In [9]:
import pandas as pd
import numpy as np

# Load the two datasets into pandas dataframes

In [10]:
data = np.loadtxt("data/HT_Sensor_dataset.dat", skiprows=1)
columns = ['id', 'time', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'temp', 'humidity']
dataset = pd.DataFrame(data, columns=columns)
dataset['id'] = dataset['id'].astype(int)

In [11]:
metadata = np.loadtxt("data/HT_Sensor_metadata.dat", skiprows=1, dtype=str)
metadata_columns = ['id', 'date', 'class', 'start_time_hours', 'duration_hours']
metadataset = pd.DataFrame(metadata, columns=metadata_columns)
metadataset['id'] = metadataset['id'].astype(int)

## Inner Join the tables to get our main dataset

In [12]:
joined_df = pd.merge(dataset, metadataset, left_on='id', right_on='id', how='inner')
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928991 entries, 0 to 928990
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                928991 non-null  int64  
 1   time              928991 non-null  float64
 2   r1                928991 non-null  float64
 3   r2                928991 non-null  float64
 4   r3                928991 non-null  float64
 5   r4                928991 non-null  float64
 6   r5                928991 non-null  float64
 7   r6                928991 non-null  float64
 8   r7                928991 non-null  float64
 9   r8                928991 non-null  float64
 10  temp              928991 non-null  float64
 11  humidity          928991 non-null  float64
 12  date              928991 non-null  object 
 13  class             928991 non-null  object 
 14  start_time_hours  928991 non-null  object 
 15  duration_hours    928991 non-null  object 
dtypes: float64(11), int6

In [13]:
## Set all records with time less than 0 equal to a background class
joined_df.loc[joined_df['time'] < 0, 'class'] = 'background'

In [14]:
X = []
y = []
## How many rows will be used for the prediction
snippet_size = 120
expected_rows_per_minute = 60
expected_snippet_time_gap = 1.05 * snippet_size / expected_rows_per_minute / 60
## How many rows will be skipped between each snippet
gap_size = 12
for start_index in range(0, len(joined_df) - snippet_size, gap_size):
    end_index = start_index + snippet_size
    ## Skip if the data spans across segments
    if joined_df.iloc[end_index,0] != joined_df.iloc[start_index,0]:
        continue
    ## Skip if the data spans across classes
    if joined_df.iloc[end_index,13] != joined_df.iloc[start_index,13]:
        continue
    ## If the time difference from beginning to end is larger than expected
    ## there must be some data missing or this chunk spans across tests
    if joined_df.iloc[end_index,1] - joined_df.iloc[start_index,1] > expected_snippet_time_gap:
        continue
    snippet = joined_df.iloc[start_index:end_index, 2:12].values
    X.append(snippet)
    y.append(joined_df.iloc[end_index,13])
    ## TRIPLE SAMPLE ALL ACTIVE POINTS TO PROMOTE HIGHER ACCURACY
    if joined_df.iloc[end_index,13] != "background":
        X.append(snippet)
        y.append(joined_df.iloc[end_index,13])
        X.append(snippet)
        y.append(joined_df.iloc[end_index,13])
        

X = np.array(X)
y = np.array(y)


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(y_train)

(113840, 120, 10)
['wine' 'banana' 'wine' ... 'banana' 'wine' 'wine']


In [17]:
from sklearn.metrics import classification_report

from sktime.classification.deep_learning.lstmfcn import LSTMFCNClassifier

model = LSTMFCNClassifier(n_epochs=100, verbose=1)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

/Users/dapomeranz/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-10-22 15:08:20.487531: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-22 15:08:20.487560: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-22 15:08:20.487565: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-22 15:08:20.487967: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-22 15:08:20.488309: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/tas

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10, 120)]            0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 10, 128)              123008    ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 10, 128)              512       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 10, 128)              0         ['batch_normalization[0][0

2023-10-22 15:08:24.182291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 15:08:24.609221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 15:08:29.040750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


890/890 [==============================] - 76s 74ms/step - loss: 0.6984 - accuracy: 0.7107
Epoch 2/100
890/890 [==============================] - 62s 70ms/step - loss: 0.4790 - accuracy: 0.8135
Epoch 3/100
890/890 [==============================] - 62s 70ms/step - loss: 0.3564 - accuracy: 0.8667
Epoch 4/100
890/890 [==============================] - 63s 71ms/step - loss: 0.2915 - accuracy: 0.8924
Epoch 5/100
890/890 [==============================] - 68s 76ms/step - loss: 0.2563 - accuracy: 0.9046
Epoch 6/100
890/890 [==============================] - 67s 75ms/step - loss: 0.2234 - accuracy: 0.9171
Epoch 7/100
890/890 [==============================] - 63s 71ms/step - loss: 0.2048 - accuracy: 0.9236
Epoch 8/100
890/890 [==============================] - 63s 71ms/step - loss: 0.1914 - accuracy: 0.9285
Epoch 9/100
890/890 [==============================] - 67s 75ms/step - loss: 0.1751 - accuracy: 0.9346
Epoch 10/100
890/890 [==============================] - 67s 76ms/step - loss: 0.1696 

Epoch 80/100
890/890 [==============================] - 62s 70ms/step - loss: 0.0651 - accuracy: 0.9737
Epoch 81/100
890/890 [==============================] - 62s 70ms/step - loss: 0.0641 - accuracy: 0.9740
Epoch 82/100
890/890 [==============================] - 63s 70ms/step - loss: 0.0621 - accuracy: 0.9752
Epoch 83/100
890/890 [==============================] - 63s 70ms/step - loss: 0.0628 - accuracy: 0.9751
Epoch 84/100
890/890 [==============================] - 63s 71ms/step - loss: 0.0629 - accuracy: 0.9749
Epoch 85/100
890/890 [==============================] - 62s 70ms/step - loss: 0.0632 - accuracy: 0.9746
Epoch 86/100
890/890 [==============================] - 64s 72ms/step - loss: 0.0616 - accuracy: 0.9757
Epoch 87/100
890/890 [==============================] - 64s 72ms/step - loss: 0.0624 - accuracy: 0.9753
Epoch 88/100
890/890 [==============================] - 63s 71ms/step - loss: 0.0624 - accuracy: 0.9748
Epoch 89/100
890/890 [==============================] - 65s 73ms

2023-10-22 16:58:39.026818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 16:58:39.302686: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


223/223 [==============================] - 5s 11ms/step
0.9709065354884048


In [18]:
model.save("trained_sktime_model")

INFO:tensorflow:Assets written to: trained_sktime_model/keras/assets


INFO:tensorflow:Assets written to: trained_sktime_model/keras/assets


<zipfile.ZipFile filename='trained_sktime_model.zip' mode='r'>